In [ ]:
!wget https://pilehvar.github.io/wic/package/WiC_dataset.zip

--2023-05-26 08:51:18--  https://pilehvar.github.io/wic/package/WiC_dataset.zip
Resolving pilehvar.github.io (pilehvar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to pilehvar.github.io (pilehvar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 275984 (270K) [application/zip]
Saving to: ‘WiC_dataset.zip’

WiC_dataset.zip     100%[===================>] 269.52K  --.-KB/s    in 0.04s   

2023-05-26 08:51:19 (6.19 MB/s) - ‘WiC_dataset.zip’ saved [275984/275984]



In [ ]:
import pandas as pd
import numpy as np
import os
import io
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
os.makedirs('WiC_data', exist_ok=True)

In [ ]:
path = 'WiC_data'

In [ ]:
dev = os.makedirs(path + '/dev', exist_ok=True )
train = os.makedirs(path + '/train', exist_ok=True )
test = os.makedirs(path+ '/test', exist_ok=True )


In [ ]:
!pip install spacy  

In [ ]:
fold = 'WiC_dataset'

In [ ]:
from zipfile import ZipFile
with ZipFile('WiC_dataset.zip', 'r') as WiC:
   WiC.extractall(fold)

In [ ]:
ds = os.listdir(fold)

In [ ]:
ds.remove('README.txt')

In [ ]:
pathss = []
for i in ds:
  pathss.append(fold +'/' + i + '/' + i+'.data.txt')


In [ ]:
path_jud = []
for i in ds:
  path_jud.append(fold + '/' + i + '/' + i+'.gold.txt')

In [ ]:
df_all = pd.DataFrame()
for i in pathss:
  df_3 = pd.read_csv(i, sep = '\t', names=["lemma", "pos", "indice", "sent1", "sent2"], encoding = 'UTF-8')
  df_3['dataset'] = i.split('/')[1]
  df_all = pd.concat([df_all, df_3])

In [ ]:
df_all_labels = pd.DataFrame()
for i in path_jud:
  tmp = pd.read_csv(i, sep = '\t', names = ['judgment'], encoding = 'UTF-8')
  tmp['dataset'] = i.split('/')[1]
  df_all_labels = pd.concat([df_all_labels, tmp])

In [ ]:
x= df_all['indice'].tolist()


In [ ]:
def create_ident(sent1, ind, lemma, dataset):
  x1 = str(ind).split('-')[0]
  id1 = lemma+'_' + x1 +'_' + sent1 + '_'+ dataset
  return id1


In [ ]:
def create_ident2(sent2, ind, lemma, dataset):
  x2 = str(ind).split('-')[1]
  id2 = lemma+'_' + x2 +'_' + sent2 + '_'+ dataset
  return id2

In [ ]:
df_ = df_all

In [ ]:
df_['id1'] = df_.apply(lambda x: create_ident(x.sent1, x.indice, x.lemma, x.dataset), axis=1)

In [ ]:
df_['id2'] = df_.apply(lambda x: create_ident2(x.sent2, x.indice, x.lemma, x.dataset), axis=1)

In [ ]:
df_judgments = df_[['id1', 'id2', 'lemma', 'dataset']]
df_judgments = df_judgments.rename(columns = {'id1':'identifier1', 'id2': 'identifier2'})

In [ ]:
df_judgments['annotator'] = 'gold'
df_judgments['comment'] = ' '
df_judgments['judgment'] = df_all_labels["judgment"].values.tolist()

In [ ]:
df_judgments = df_judgments[["identifier1", "identifier2", "annotator", "judgment", "comment", "lemma", 'dataset']]

In [ ]:
df_dash = df_[['lemma', 'sent1', 'sent2', 'dataset', 'id1', 'id2']]

In [ ]:
df1 = df_dash[['lemma', 'sent1', 'dataset', 'id1']]
df1.columns =  ['word', 'sent', 'dataset', 'id']
#df1["index"] = df1.groupby("word").cumcount()*2

In [ ]:
df2 =  df_dash[['lemma', 'sent2', 'dataset', 'id2']]
df2.columns =  ['word', 'sent', 'dataset', 'id']
#df2["index"] = df2.groupby("word").cumcount()*2+1

In [ ]:
df_final = pd.concat([df1, df2])
df_final= df_final.sort_values(by = ['word'], ascending = True) 

In [ ]:
dftodict = pd.unique(df_final[['sent', 'id']].values.ravel())

In [ ]:
result_df = df_final.drop_duplicates()

In [ ]:
sent_to_id = pd.Series(result_df.id.values,index=result_df.sent).to_dict()

In [ ]:
ind1 = []
ind2 = []
for i in x:
    ind1.append(i.split('-')[0])
    ind2.append(i.split('-')[1])

In [ ]:
df_all.loc[:,'ind1'] = ind1
df_all.loc[:,'ind2'] = ind2


In [ ]:
df1 = df_all[['lemma', 'pos', 'sent1', 'ind1', 'dataset']]
df1.columns =  ['lemma','pos', 'context_tokenized','ind', 'dataset']


In [ ]:
df2 = df_all[['lemma', 'pos', 'sent2', 'ind2', 'dataset']]
df2.columns =  ['lemma','pos', 'context_tokenized','ind', 'dataset']

In [ ]:
df_final_uses = pd.concat([df1, df2])
df_final_uses = df_final_uses.sort_values(by = ['lemma'], ascending = True)


In [ ]:
df_final_uses = df_final_uses.reset_index(drop =True)

In [ ]:
df_final_uses = df_final_uses.drop_duplicates()

In [ ]:
def get_indice(sent, indice):
  tag = ''
  indice = int(indice)
  tok = sent.split(" ")
  #print(tok)
  for i in range(len(tok)):
    if i == indice:
      tag = tok[i]
  return str(sent.find(tag))+":"+str(sent.find(tag)+len(tag))

In [ ]:
df_final_uses["indexes_target_token"] = df_final_uses.apply(lambda x: get_indice(x.context_tokenized, x.ind), axis=1)


In [ ]:
def get_indices_of_sent(sentence):
    return "0:"+str(len(sentence))

In [ ]:
df_final_uses["indexes_target_sentence"] = df_final_uses["context_tokenized"].apply(get_indices_of_sent)

In [ ]:
def get_len_tok(sentence):
  return "0:"+str(len(sentence.split(" ")))

In [ ]:
df_final_uses['indexes_target_sentence_tokenized'] = df_final_uses["context_tokenized"].apply(get_len_tok)


In [ ]:
lem = ''
def lemmatizr(sent):
  doc = nlp(sent)
  for i in doc:
    lem += i.lemma_
    return lem

In [ ]:
df_final_uses['context_lemmatized'] = df_final_uses["context_tokenized"].apply(lambda row: " ".join([w.lemma_ for w in nlp(row)]))

In [ ]:
df_final_uses["indice"] = df_final_uses.groupby("lemma").cumcount()

In [ ]:
df_final_uses['context_pos'] = df_final_uses["context_tokenized"].apply(lambda row: " ".join([w.pos_ for w in nlp(row)]))

In [ ]:
df_final_uses['date'] = " "
df_final_uses['grouping'] = 1
df_final_uses['description'] = " "
df_final_uses['context'] = df_final_uses['context_tokenized']
#df_final_dev_uses['identifier'] = ""
df_final_uses['indexes_target_token_tokenized'] = df_final_uses['ind']


In [ ]:
def get_identifier(sent, dictionar):
  for k, v in sent_to_id.items():
    for i in v:
      if sent == k:
        return v

In [ ]:
sents = df_final_uses.context.tolist()

In [ ]:
aggre = []
for i in sents:
  aggre.append(get_identifier(i, sent_to_id))

In [ ]:
df_final_uses['identifier'] = aggre

In [ ]:
final_dfs = df_final_uses[['lemma', 'pos', 'date', 'grouping', 'identifier', 'description', 'context', 'indexes_target_token', 'indexes_target_sentence', 'context_tokenized', 'indexes_target_token_tokenized', 'indexes_target_sentence_tokenized', 'context_lemmatized', 'context_pos', 'dataset']]


In [ ]:
final_df = final_dfs.reset_index(drop= True)

In [ ]:
!git clone https://github.com/Garrafao/WUGs.git

Cloning into 'WUGs'...
remote: Enumerating objects: 1402, done.
remote: Counting objects: 100% (328/328), done.
remote: Compressing objects: 100% (166/166), done.
remote: Total 1402 (delta 184), reused 274 (delta 149), pack-reused 1074
Receiving objects: 100% (1402/1402), 1.80 MiB | 3.24 MiB/s, done.
Resolving deltas: 100% (862/862), done.


In [ ]:
os.makedirs('/content/WUGs/scripts/misc/data/dwug_en/data/*/', exist_ok=True )

In [ ]:
import csv
final_df.to_csv('/content/WUGs/scripts/misc/data/dwug_en/data/*/uses.csv', sep='\t', encoding='utf-8', quoting = csv.QUOTE_NONE, quotechar='')


In [ ]:
%run /content/WUGs/scripts/misc/use2normalize.py /content/WUGs/scripts/misc/data/dwug_en/data/*/uses.csv dwug_en output

Streaming output truncated to the last 5000 lines.
target:  make
target:  make
target:  make
target:  Make
target:  made
target:  Make
target:  make
target:  make
target:  make
target:  makes
target:  makes
target:  make
target:  make
target:  make
target:  Make
target:  made
target:  made
target:  made
target:  Make
target:  make
target:  Make
target:  Make
target:  Make
target:  Make
target:  Make
target:  Make
target:  make
target:  make
target:  Make
target:  Make
target:  Make
target:  made
target:  Make
target:  makes
target:  make
target:  makes
target:  Make
target:  made
target:  make
target:  made
target:  make
target:  Make
target:  make
target:  Make
target:  Make
target:  Make
target:  Make
target:  Make
target:  Make
target:  Make
target:  made
target:  make
target:  Make
target:  made
target:  make
target:  made
target:  make
target:  made
target:  made
target:  makeup
target:  makeup
target:  making
target:  making
target:  men
target:  manning
target:  man
target:  man

In [ ]:
final_uses = pd.read_csv('/content/output', delimiter = '\t',quoting =3)


In [ ]:
final_uses['context_tokenized'] = final_df['context_tokenized']
final_uses['pos'] = final_df['pos']
final_uses['dataset'] = final_df['dataset']
final_uses['grouping'] = final_df['grouping']
final_uses['lemma'] = final_df['lemma']
final_uses['indexes_target_token_tokenized'] = final_df['indexes_target_token_tokenized']
final_uses['indexes_target_sentence_tokenized'] = final_df['indexes_target_sentence_tokenized']
final_uses['date'] = " "
final_uses['description'] = " "
final_uses['context_lemmatized'] = final_df['context_lemmatized']
final_uses['context_pos'] = final_df['context_pos']


In [ ]:
final_uses = final_uses[['lemma', 'pos', 'date', 'grouping', 'identifier', 'description', 'context', 'indexes_target_token', 'indexes_target_sentence', 'context_tokenized', 'indexes_target_token_tokenized', 'indexes_target_sentence_tokenized', 'context_lemmatized', 'context_pos', 'dataset']]


In [ ]:
for i in list(final_uses["dataset"].value_counts().index):
    df_temp = final_uses[final_uses["dataset"]==i]
    if not os.path.exists(i):
        os.mkdir(i)
    df_temp.to_csv(path +'/'+ i +'/uses.csv',index = False, sep='\t', encoding='utf-8', quoting=csv.QUOTE_NONE, quotechar = '')

In [ ]:
for i in list(df_judgments["dataset"].value_counts().index):
    df_temp = df_judgments[df_judgments["dataset"]==i]
    if not os.path.exists(i):
        os.mkdir(i)
    df_temp.to_csv(path + '/' + i +'/judgments.csv',index = False, sep='\t', encoding='utf-8', quoting=csv.QUOTE_NONE, quotechar = '')

In [ ]:
df_u = pd.read_csv('WiC_data/train/uses.csv', delimiter = '\t', quoting =3)

In [ ]:
df_j = pd.read_csv('WiC_data/train/judgments.csv', delimiter = '\t', quoting =3)

In [ ]:
os.makedirs('outputs-train', exist_ok=True)

In [ ]:
outpath = 'outputs-train/'

In [ ]:
for i in list(df_u["lemma"].value_counts().index):
    df_temp = df_u[df_u["lemma"]==i]
    if not os.path.exists(outpath+i):
        os.mkdir(outpath + i)
    df_temp.to_csv(outpath+i +'/uses.csv',index = False, sep='\t', encoding='utf-8', quoting=csv.QUOTE_NONE, quotechar = '')

In [ ]:
for i in list(df_j["lemma"].value_counts().index):
    df_temp = df_j[df_j["lemma"]==i]
    if not os.path.exists(outpath+i):
        os.mkdir(outpath + i)
    df_temp.to_csv(outpath+i +'/judgments.csv',index = False, sep='\t', encoding='utf-8', quoting=csv.QUOTE_NONE, quotechar = '')

In [ ]:
!zip -r /content/outputs-train.zip /content/outputs-train

  adding: content/outputs-train/ (stored 0%)
  adding: content/outputs-train/separation/ (stored 0%)
  adding: content/outputs-train/separation/judgments.csv (deflated 38%)
  adding: content/outputs-train/separation/uses.csv (deflated 64%)
  adding: content/outputs-train/assign/ (stored 0%)
  adding: content/outputs-train/assign/judgments.csv (deflated 36%)
  adding: content/outputs-train/assign/uses.csv (deflated 65%)
  adding: content/outputs-train/ring/ (stored 0%)
  adding: content/outputs-train/ring/judgments.csv (deflated 35%)
  adding: content/outputs-train/ring/uses.csv (deflated 67%)
  adding: content/outputs-train/loan/ (stored 0%)
  adding: content/outputs-train/loan/judgments.csv (deflated 31%)
  adding: content/outputs-train/loan/uses.csv (deflated 64%)
  adding: content/outputs-train/twinkle/ (stored 0%)
  adding: content/outputs-train/twinkle/judgments.csv (deflated 35%)
  adding: content/outputs-train/twinkle/uses.csv (deflated 63%)
  adding: content/outputs-train/cuddl

In [ ]:
from google.colab import files
files.download("/content/outputs-train.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!zip -r /WiC_data.zip /content/WiC_data

  adding: content/WiC_data/ (stored 0%)
  adding: content/WiC_data/test/ (stored 0%)
  adding: content/WiC_data/test/judgments.csv (deflated 65%)
  adding: content/WiC_data/test/uses.csv (deflated 80%)
  adding: content/WiC_data/dev/ (stored 0%)
  adding: content/WiC_data/dev/judgments.csv (deflated 64%)
  adding: content/WiC_data/dev/uses.csv (deflated 80%)
  adding: content/WiC_data/train/ (stored 0%)
  adding: content/WiC_data/train/judgments.csv (deflated 68%)
  adding: content/WiC_data/train/uses.csv (deflated 81%)


In [ ]:
from google.colab import files
files.download("/WiC_data.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>